In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet152, ResNet152V2
from tensorflow.keras.optimizers import SGD
import wandb
from wandb.keras import WandbCallback

# Loading Data Set
Three sets are created: training, validation, and test. 
- Labels are generated based on the folder structure. Class name must correspond to the subfolder name.
- Loading in batches, of size 32, to reduce memory usage.
- Label mode is set to categorical, which means that the labels are encoded as a categorical vector.

Bilinear interploation is set to default. This specify the method used in the resizing procedure. By default aspect ratio is not perserved, i.e., the ratio between image width and height.

One hot encoding is utilized when label mode is set to categorical.


The image load documentation is available [here](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) and an example is available [here](https://keras.io/api/data_loading/image/).

In [3]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )

Found 297792 files belonging to 12 classes.
Found 37219 files belonging to 12 classes.


In [5]:
# list of callbacks
callbacks = [
            WandbCallback(mode="min", monitor="val_loss", save_graph=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, mode="min"),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode="min"),
          ]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


# ResNet152

In [4]:
# initialize wandb
run = wandb.init(project="ResNet152", config={"learning_rate": 0.1,
                                               "epochs": 100, 
                                               "momentum": 0.9,
                                               "batch_size": 32,
                                               "input_shape": (224, 224, 3),
                                               "optimizer": "Adam",
                                               "loss": "categorical_crossentropy",
                                               "metrics": ["accuracy"],
                                               "verbose": 1,
                                               "name": "ResNet152",
                                               "architecture": "ResNet152"
                                              })

# configs
cfg = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gabri-torland (nubs). Use `wandb login --relogin` to force relogin


## Build the model
The hyperparameters and fully connected layers closely resemble those employed in the 2015 paper that first presented residual networks. The paper is available here: http://arxiv.org/abs/1512.03385

In [9]:
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=cfg.input_shape)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
predictions = layers.Dense(len(train_ds.class_names), activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

12


## Compile the model

In [14]:
model.compile(optimizer=SGD(learning_rate=cfg.learning_rate, momentum=cfg.momentum),
              loss=cfg.loss,
              metrics=cfg.metrics)

## Train the model

In [15]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=cfg.batch_size, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

Epoch 1/100


2023-04-06 14:15:33.707359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [37219]
	 [[{{node Placeholder/_4}}]]
2023-04-06 14:15:33.707509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [37219]
	 [[{{node Placeholder/_4}}]]
2023-04-06 14:15:33.737485: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2977

   6/9306 [..............................] - ETA: 10:13:39 - loss: 5.5638 - accuracy: 0.2083

KeyboardInterrupt: 

In [ ]:
wandb.finish()

# ResNet152V2

In [ ]:
## initialize wandb
run = wandb.init(project="ResNet152V2", config={"learning_rate": 0.1,
                                               "epochs": 100, 
                                               "momentum": 0.9,
                                               "batch_size": 32,
                                               "input_shape": (224, 224, 3),
                                               "optimizer": "Adam",
                                               "loss": "categorical_crossentropy",
                                               "metrics": ["accuracy"],
                                               "verbose": 1,
                                               "name": "ResNet152V2",
                                               "architecture": "ResNet152V2"
                                              })

# configs
cfg = wandb.config

## Build the model
This model draws upon the architecture delineated in the 2016 paper (http://arxiv.org/abs/1603.05027). Nonetheless, the hyperparameters and fully connected layers remain consistent with those found in the 2015 papers. Therefore, the hyperparameters and fully connected layers in this model remain unaltered.

In [ ]:
base_model = models.ResNet152V2(weights='imagenet', include_top=False, input_shape=cfg.input_shape)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
predictions = layers.Dense(len(train_ds.class_names), activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

## Compile the model

In [ ]:
model.compile(optimizer=SGD(learning_rate=cfg.learning_rate, momentum=cfg.momentum),
              loss=cfg.loss,
              metrics=cfg.metrics)

## Train the model

In [ ]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=cfg.batch_size, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

In [ ]:
wandb.finish()